In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[pillow, tqdm]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'monai'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.2 MB/s eta 0:00:00


In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
from sklearn.metrics import classification_report
from monai.networks.nets import DenseNet121
from monai.transforms import Compose, AddChannel, ScaleIntensity, ToTensor
import os
import numpy as np
from monai.metrics import ROCAUCMetric
# data transformations
transform = Compose([
    AddChannel(),
    ScaleIntensity(),
    ToTensor()
])

# Custom torchvision transform
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Lambda(lambda x: np.expand_dims(x, axis=0)),  # Add channel dimension
# ])
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # transforms.Lambda(lambda x: x[None, :, :]),  # Add channel dimension
])

# Loading the MNIST dataset
# train_dataset = MNIST(root='./', train=True, download=True, transform=transform)
auc_metric = ROCAUCMetric()
data_dir = "/content/MNIST"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


train_dataset = MNIST(root='./', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./', train=False, download=True, transform=transform)

# create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# initialize the model
model = DenseNet121(spatial_dims=2, in_channels=1, out_channels=10)
model = model.to(device)

# define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_epoch(loader, model, criterion, optimizer):
    model.train()
    for batch, (images, labels) in enumerate(loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def test_epoch(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch, (images, labels) in enumerate(loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Train the model
train_epoch(train_loader, model, criterion, optimizer)

# Test the model
accuracy = test_epoch(test_loader, model)
print(f'Accuracy of the model on test images: {accuracy * 100:.2f}%')

# classification report
y_true = []
y_pred = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

print(classification_report(y_true, y_pred))


/usr/local/lib/python3.10/dist-packages/monai/utils/deprecate_utils.py:111: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
  warn_deprecated(obj, msg, warning_category)


100%|██████████| 9912422/9912422 [00:00<00:00, 149384725.99it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 110123358.02it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 67008607.74it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 21947613.79it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Accuracy of the model on test images: 98.51%
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       980
           1       0.99      1.00      0.99      1135
           2       0.98      1.00      0.99      1032
           3       0.99      0.99      0.99      1010
           4       1.00      0.95      0.97       982
           5       0.99      0.98      0.99       892
           6       0.97      0.99      0.98       958
           7       0.99      0.99      0.99      1028
           8       0.99      0.99      0.99       974
           9       0.97      0.97      0.97      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.98      0.98     10000
weighted avg       0.99      0.99      0.99     10000

